In [ ]:
# Packages
import os
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import win32com.client as wc
from datetime import datetime

# Constants
FILE_PATH = ""
SQL_SCRIPT = ""
OUTPUT_PATH = ""

# Excel
file_name = os.path.basename(FILE_PATH)
version = "1.0.0.1"
title = ""
subject = ""
author = ""
company = ""
category = ""
keywords = ""
comments = ""
status = ""

# Email
mail_from = ""
mail_to = ""
mail_cc = ""
mail_body = ""
email_subject = ""
mail_attachment = ""

# Days
today = datetime.today()
current_day = today.weekday()
weekend = 3
weekday = 1

# Functions
# Load environment variables
def load_env_vars():
    current_directory = Path(__file__).resolve().parent if "__file__" in locals() else Path.cwd()
    environment_variables = current_directory / ".env"
    load_dotenv(environment_variables)

# Create database connection
def create_db_connection():
    USER = os.getenv("USER")
    PASSWORD = os.getenv("PASSWORD")
    HOST = os.getenv("HOST")
    PORT = os.getenv("PORT")
    DATABASE = os.getenv("DATABASE")
    connection_string = f"oracle+oracledb://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"

    engine = create_engine(connection_string)
    try:
        connection = engine.connect()
        if connection:
            print(f"{file_name} - Connection established.")
            return connection
        else:
            print(f"{file_name} - Connection failed.")
            return None
    except Exception as error:
        print(f"{file_name} - Error: {error}")
        return None
    
# Read SQL file
def read_sql_file(file_path):
    with open(file_path) as file:
        return text(file.read())
    
# Execute query
def execute_query(connection, query, params):
    data = pd.read_sql(query, connection, params=params)
    return data

# Write to Excel
def write_to_excel(data, output_path):
    output_path = OUTPUT_PATH
    with pd.ExcelWriter(
        output_path,
        engine="xlsxwriter",
        datetime_format="dd mmm yyyy hh:mm:ss",
        date_format="dd mmm yyyy",
    ) as writer:
        data.to_excel(writer, index=False, sheet_name="Sheet1")
        workbook = writer.book
        workbook.set_properties = {
            "Title": title,
            "Subject": subject,
            "Author": author,
            "Company": company,
            "Category": category,
            "Keywords": keywords,
            "Comments": comments,
            "Status": status,
        }
        print(f"{file_name} - Excel file created.")

# Send email
def send_email(subject):
    outlook = wc.Dispatch("Outlook.Application")
    mail = outlook.CreateItem(0)
    mail.To = mail_to
    mail.CC = mail_cc
    mail.Subject = email_subject
    mail.Body = mail_body
    for attachment in mail_attachment:
        mail.Attachments.Add(attachment)
    main.Display()
    # mail.Send()
    print(f"{file_name} - Email sent.")

# Main
def main():
    print(f"{file_name} - Started.")
    load_env_vars()
    try:
        connection = create_db_connection()
        query = read_sql_file(FILE_PATH)
        if current_day == 0:
            params = {"days": weekend}
            data = execute_query(connection, query, params)
            write_to_excel(data, OUTPUT_PATH)
            send_email(subject)
        elif current_day in (1, 2, 3, 4):
            params = {"days": weekday}
            data = execute_query(connection, query, params)
            write_to_excel(data, OUTPUT_PATH)
            send_email(subject)
        else:
            print(f"{file_name} - Is not scheduled to run today.")
    except Exception as error:
        print(f"{file_name} - Error: {error}")
    print(f"{file_name} - Finished.")

if __name__ == "__main__":
    main()